# 🌍 Cross-Platform Chords Notebook

This notebook works on Windows, macOS, Linux, and WSL2!

In [1]:
import platform
import os

print("🌍 Platform Detection:")
print("=" * 30)

system = platform.system().lower()
print(f"OS: {platform.system()}")

if system == "darwin":
    print("✅ macOS - will use CoreAudio")
elif system == "linux":
    if os.path.exists("/dev/snd"):
        print("✅ Linux - will use ALSA")
    else:
        print("⚠️ WSL2 - will use PulseAudio")
elif system == "windows":
    print("✅ Windows - will use DirectSound")
else:
    print("⚠️ Unknown OS - will try PulseAudio")

🌍 Platform Detection:
OS: Darwin
✅ macOS - will use CoreAudio


In [4]:
import fluidsynth
import time
import platform
import os

# Smart driver selection
def get_driver():
    system = platform.system().lower()
    if system == "darwin": return "coreaudio"
    elif system == "linux":
        return "alsa" if os.path.exists("/dev/snd") else "pulseaudio"
    elif system == "windows": return "dsound"
    else: return "pulseaudio"

# Get best driver
driver = get_driver()
print(f"🎯 Using {driver} driver")

# Try to play audio
try:
    sf2 = "./piano.sf2"
    fs = fluidsynth.Synth()
    fs.start(driver=driver)
    print(f"✅ Started with {driver}")
    
    # Load and play
    sfid = fs.sfload(sf2)
    fs.program_select(0, sfid, 0, 0)
    
    notes = [69, 73, 76]  # A major triad
    vel = 96
    
    print("🎵 Playing chord...")
    for n in notes:
        fs.noteon(0, n, vel)
    
    time.sleep(2.5)
    
    for n in notes:
        fs.noteoff(0, n)
    
    fs.delete()
    print("✅ Success!")
    
except Exception as e:
    print(f"⚠️ Audio failed: {e}")
    print("🔄 Creating MIDI file instead...")
    
    try:
        import mido
        mid = mido.MidiFile()
        track = mido.MidiTrack()
        mid.tracks.append(track)
        
        track.append(mido.MetaMessage("set_tempo", tempo=mido.bpm2tempo(120)))
        
        for note in notes:
            track.append(mido.Message("note_on", note=note, velocity=vel, time=0))
        
        track.append(mido.Message("note_off", note=notes[0], velocity=0, time=4800))
        track.append(mido.Message("note_off", note=notes[1], velocity=0, time=0))
        track.append(mido.Message("note_off", note=notes[2], velocity=0, time=0))
        
        mid.save("chord_output.mid")
        print("✅ MIDI file created - works everywhere!")
        
    except Exception as midi_error:
        print(f"❌ MIDI failed: {midi_error}")

🎯 Using coreaudio driver
✅ Started with coreaudio
🎵 Playing chord...
✅ Success!
